Playground for testing market analysis and brokerage API connections. see market analyzer 2.0 for cleaner implementation

In [5]:
#no need to run cell, was used for a 1-time scrape to get the 50 largerst publicly traded restaurants by market cap and now tickers are hardcoded.
#Can be run again to update list if things change
"""import requests
from bs4 import BeautifulSoup


TARGET_URL = f"https://companiesmarketcap.com/restaurant-chains/largest-restaurant-chain-companies-by-market-cap/"

# make an HTTP request
page = requests.get(TARGET_URL)


# use an HTML parser to grab the content from "page"
soup = BeautifulSoup(page.content, "html.parser")


# get the items that describe the stock
items = soup.find_all("div", {"class": "company-code"})
ticker_symbols = [code.text.strip() for code in items]"""
ticker_symbols = ['MCD', 'CMG', 'YUM', 'QSR', 'DRI', 'YUMC', 'DPZ', 'WING', '6862.HK', 'TXRH', 'AMR.AE', 'CAVA', '2702.T', 'JBFCF', 'SHAK', 'WEN', 'DMP.AX', 'PLAY', 'ARCO', 'BLMN', 'PZZA', 'DEVYANI.NS', 'SSPG.L', 'EAT', 'CAKE', 'SG', 'FWRG', 'JACK', 'EAT.MC', 'KRUS', 'MTY.TO', 'BJRI', 'PTLO', '9658.HK', 'DIN', '0341.HK', 'RTN.L', 'CHUY', 'BH', 'TAST', 'DENN', 'PBPB', '2753.TW', 'PZA.TO', 'LOCO', 'NATH', 'BARBEQUE.NS', 'GENK', 'RBD.NZ', 'THCH', 'FAT', '0052.HK', 'STKS', 'RRGB', 'NDLS', 'RAVE']

In [4]:
!pip install oandapyV20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for oandapyV20: filename=oandapyV20-0.7.2-py3-none-any.whl size=69779 sha256=2d2871be640077e7af4224a4894867f5e4946ab9a66ecf22d7e4858a925e1737
  Stored in directory: /root/.cache/pip/wheels/8e/69/ab/a6da84a85b9bf3b5a98ca15c6c52b7854f32b10c70fe0531a1
Successfully built oandapyV20


In [6]:
import oandapyV20
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
import json

# Initialize the API client
# Access token is for a demo trading account, DO NOT hardcode tokens for real money accounts, use a secure configuration method
client = oandapyV20.API(access_token='8e00ca0c834c14f4855707d113b8625e-66f48575bc06e2a7621f83dae68c7607')

# Fetch account details
# Demo account_id
account_id = '101-001-28514425-001'
r = accounts.AccountDetails(account_id)
account_details = client.request(r)
print(account_details)

# Test connection: Place a market order to buy 1 Euro
data = {
    "order": {
        "type": "MARKET",  # Specifies the type of order
        "instrument": "EUR_USD",
        "units": "1",  # Positive for buy, negative for sell
        "timeInForce": "FOK",  # Fills or kills the order
        "positionFill": "DEFAULT"
    }
}

# Create and send the order request
order = orders.OrderCreate(accountID=account_id, data=data)
response = client.request(order)

print(json.dumps(response, indent=2))


{'account': {'guaranteedStopLossOrderMode': 'DISABLED', 'hedgingEnabled': False, 'id': '101-001-28514425-001', 'createdTime': '2024-03-03T18:28:56.625100678Z', 'currency': 'USD', 'createdByUserID': 28514425, 'alias': 'Primary', 'marginRate': '0.02', 'lastTransactionID': '12', 'balance': '99999.9854', 'openTradeCount': 3, 'openPositionCount': 1, 'pendingOrderCount': 0, 'pl': '0.0000', 'resettablePL': '0.0000', 'resettablePLTime': '0', 'financing': '-0.0146', 'commission': '0.0000', 'dividendAdjustment': '0', 'guaranteedExecutionFees': '0.0000', 'orders': [], 'positions': [{'instrument': 'EUR_USD', 'long': {'units': '300', 'averagePrice': '1.08569', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '-0.0146', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.0000', 'tradeIDs': ['7', '9', '12'], 'unrealizedPL': '-0.3260'}, 'short': {'units': '0', 'pl': '0.0000', 'resettablePL': '0.0000', 'financing': '0.0000', 'dividendAdjustment': '0.0000', 'guaranteedExecutionFees': '0.

In [28]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr

yf.pdr_override()

# For time stamps
from datetime import datetime


end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)


In [31]:
#create list of restaurant ticker symbols called ticker symbols
ticker_list = ['MCD', 'CMG', 'YUM', 'QSR', 'DRI', 'YUMC', 'DPZ', 'WING', '6862.HK', 'TXRH', 'CAVA', '2702.T', 'JBFCF', 'SHAK', 'WEN', 'DMP.AX', 'PLAY', 'ARCO', 'BLMN', 'PZZA', 'DEVYANI.NS', 'SSPG.L', 'EAT', 'CAKE', 'SG', 'FWRG', 'JACK', 'EAT.MC', 'KRUS', 'MTY.TO', 'BJRI', 'PTLO', '9658.HK', 'DIN', '0341.HK', 'CHUY', 'BH', 'TAST', 'DENN', 'PBPB', '2753.TW', 'PZA.TO', 'LOCO', 'NATH', 'BARBEQUE.NS', 'GENK', 'RBD.NZ', 'THCH', 'FAT', '0052.HK', 'STKS', 'RRGB', 'NDLS', 'RAVE']
#removed 'AMR.AE' and 'RTN.L'
all_data = pd.DataFrame()
# Define a function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Define a function to calculate SMA
def calculate_sma(data, window=20):
    return data.rolling(window=window).mean()

for ticker in ticker_list:
    # Fetch the daily data
    daily_data = yf.download(ticker, start=start, end=end)[['Close', 'Volume']].copy()
    daily_data.reset_index(inplace=True)

    daily_data['RSI'] = calculate_rsi(daily_data['Close'])
    daily_data['SMA_20'] = calculate_sma(daily_data['Close'])

    # Add lagged features for 'Close' and 'Volume'
    daily_data['Close_lag1'] = daily_data['Close'].shift(1)
    daily_data['Volume_lag1'] = daily_data['Volume'].shift(1)

    # Fetch the market cap and target price (mean)
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.info
    market_cap = info.get('marketCap', None)
    target_mean_price = info.get('targetMeanPrice', None)

    # Add the ticker, market cap, and target mean price to the daily data
    daily_data['Ticker'] = ticker
    daily_data['Market_Cap'] = market_cap
    daily_data['Target_Mean_Price'] = target_mean_price

    # Append to the main DataFrame
    all_data = pd.concat([all_data, daily_data])

# Convert 'Ticker' column to one-hot encoded columns
#all_data = pd.get_dummies(all_data, columns=['Ticker'])

print(all_data.shape)
print(all_data.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

(13363, 10)
        Date       Close   Volume  RSI  SMA_20  Close_lag1  Volume_lag1  \
0 2023-03-06  270.640015  2217600  NaN     NaN         NaN          NaN   
1 2023-03-07  267.130005  2766600  NaN     NaN  270.640015    2217600.0   
2 2023-03-08  265.329987  2314000  NaN     NaN  267.130005    2766600.0   
3 2023-03-09  261.630005  2339300  NaN     NaN  265.329987    2314000.0   
4 2023-03-10  262.029999  3093100  NaN     NaN  261.630005    2339300.0   

  Ticker    Market_Cap  Target_Mean_Price  
0    MCD  210073796608             327.05  
1    MCD  210073796608             327.05  
2    MCD  210073796608             327.05  
3    MCD  210073796608             327.05  
4    MCD  210073796608             327.05  


In [32]:
print(all_data.columns)

Index(['Date', 'Close', 'Volume', 'RSI', 'SMA_20', 'Close_lag1', 'Volume_lag1',
       'Ticker', 'Market_Cap', 'Target_Mean_Price'],
      dtype='object')


In [33]:
# Step 1: Feature Preparation
# Drop rows with missing values to simplify the example
all_data.dropna(inplace=True)

# Step 2: Label Creation
# Shift the 'Close' price to create the target variable: 1 if the price goes up, else 0
all_data['Target'] = (all_data.groupby('Ticker')['Close'].shift(-1) > all_data['Close']).astype(int)

# Filter out the last row for each ticker since it won't have a label
all_data = all_data.groupby('Ticker').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)

all_data = pd.get_dummies(all_data, columns=['Ticker'])

# Assuming we are focusing on one stock for simplicity, filter by ticker if desired
# all_data = all_data[all_data['Ticker'] == 'AAPL']

# Separate features and target variable
X = all_data[['RSI', 'SMA_20', 'Close_lag1', 'Volume_lag1']]
y = all_data['Target']

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Model Training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5122067975107707


In [24]:

# URL of the Wikipedia page containing the list of S&P 500 companies
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Use pandas to read the table from the webpage
sp500_table = pd.read_html(url, attrs={"id": "constituents"})[0]

# The DataFrame 'sp500_table' now contains the list of S&P 500 companies
sp500_symbols = sp500_table['Symbol'].tolist()

# Optionally, extract other information such as the industry sector
sp500_sectors = sp500_table[['Symbol', 'GICS Sector']].set_index('Symbol').to_dict()['GICS Sector']

#print(sp500_symbols)  # Print the first 10 ticker symbols to verify
print(list(sp500_sectors.items())[:10])  # Print the sector information for the first 10 companies
sp500_symbols_hardcoded = ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BK', 'BBWI', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'TECH', 'BIIB', 'BLK', 'BX', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CHRW', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DVA', 'DAY', 'DE', 'DAL', 'XRAY', 'DVN', 'DXCM', 'FANG', 'DLR', 'DFS', 'DG', 'DLTR', 'D', 'DPZ', 'DOV', 'DOW', 'DHI', 'DTE', 'DUK', 'DD', 'EMN', 'ETN', 'EBAY', 'ECL', 'EIX', 'EW', 'EA', 'ELV', 'LLY', 'EMR', 'ENPH', 'ETR', 'EOG', 'EPAM', 'EQT', 'EFX', 'EQIX', 'EQR', 'ESS', 'EL', 'ETSY', 'EG', 'EVRG', 'ES', 'EXC', 'EXPE', 'EXPD', 'EXR', 'XOM', 'FFIV', 'FDS', 'FICO', 'FAST', 'FRT', 'FDX', 'FIS', 'FITB', 'FSLR', 'FE', 'FI', 'FLT', 'FMC', 'F', 'FTNT', 'FTV', 'FOXA', 'FOX', 'BEN', 'FCX', 'GRMN', 'IT', 'GEHC', 'GEN', 'GNRC', 'GD', 'GE', 'GIS', 'GM', 'GPC', 'GILD', 'GPN', 'GL', 'GS', 'HAL', 'HIG', 'HAS', 'HCA', 'PEAK', 'HSIC', 'HSY', 'HES', 'HPE', 'HLT', 'HOLX', 'HD', 'HON', 'HRL', 'HST', 'HWM', 'HPQ', 'HUBB', 'HUM', 'HBAN', 'HII', 'IBM', 'IEX', 'IDXX', 'ITW', 'ILMN', 'INCY', 'IR', 'PODD', 'INTC', 'ICE', 'IFF', 'IP', 'IPG', 'INTU', 'ISRG', 'IVZ', 'INVH', 'IQV', 'IRM', 'JBHT', 'JBL', 'JKHY', 'J', 'JNJ', 'JCI', 'JPM', 'JNPR', 'K', 'KVUE', 'KDP', 'KEY', 'KEYS', 'KMB', 'KIM', 'KMI', 'KLAC', 'KHC', 'KR', 'LHX', 'LH', 'LRCX', 'LW', 'LVS', 'LDOS', 'LEN', 'LIN', 'LYV', 'LKQ', 'LMT', 'L', 'LOW', 'LULU', 'LYB', 'MTB', 'MRO', 'MPC', 'MKTX', 'MAR', 'MMC', 'MLM', 'MAS', 'MA', 'MTCH', 'MKC', 'MCD', 'MCK', 'MDT', 'MRK', 'META', 'MET', 'MTD', 'MGM', 'MCHP', 'MU', 'MSFT', 'MAA', 'MRNA', 'MHK', 'MOH', 'TAP', 'MDLZ', 'MPWR', 'MNST', 'MCO', 'MS', 'MOS', 'MSI', 'MSCI', 'NDAQ', 'NTAP', 'NFLX', 'NEM', 'NWSA', 'NWS', 'NEE', 'NKE', 'NI', 'NDSN', 'NSC', 'NTRS', 'NOC', 'NCLH', 'NRG', 'NUE', 'NVDA', 'NVR', 'NXPI', 'ORLY', 'OXY', 'ODFL', 'OMC', 'ON', 'OKE', 'ORCL', 'OTIS', 'PCAR', 'PKG', 'PANW', 'PARA', 'PH', 'PAYX', 'PAYC', 'PYPL', 'PNR', 'PEP', 'PFE', 'PCG', 'PM', 'PSX', 'PNW', 'PXD', 'PNC', 'POOL', 'PPG', 'PPL', 'PFG', 'PG', 'PGR', 'PLD', 'PRU', 'PEG', 'PTC', 'PSA', 'PHM', 'QRVO', 'PWR', 'QCOM', 'DGX', 'RL', 'RJF', 'RTX', 'O', 'REG', 'REGN', 'RF', 'RSG', 'RMD', 'RVTY', 'RHI', 'ROK', 'ROL', 'ROP', 'ROST', 'RCL', 'SPGI', 'CRM', 'SBAC', 'SLB', 'STX', 'SRE', 'NOW', 'SHW', 'SPG', 'SWKS', 'SJM', 'SNA', 'SO', 'LUV', 'SWK', 'SBUX', 'STT', 'STLD', 'STE', 'SYK', 'SYF', 'SNPS', 'SYY', 'TMUS', 'TROW', 'TTWO', 'TPR', 'TRGP', 'TGT', 'TEL', 'TDY', 'TFX', 'TER', 'TSLA', 'TXN', 'TXT', 'TMO', 'TJX', 'TSCO', 'TT', 'TDG', 'TRV', 'TRMB', 'TFC', 'TYL', 'TSN', 'USB', 'UBER', 'UDR', 'ULTA', 'UNP', 'UAL', 'UPS', 'URI', 'UNH', 'UHS', 'VLO', 'VTR', 'VLTO', 'VRSN', 'VRSK', 'VZ', 'VRTX', 'VFC', 'VTRS', 'VICI', 'V', 'VMC', 'WRB', 'WAB', 'WBA', 'WMT', 'DIS', 'WBD', 'WM', 'WAT', 'WEC', 'WFC', 'WELL', 'WST', 'WDC', 'WRK', 'WY', 'WHR', 'WMB', 'WTW', 'GWW', 'WYNN', 'XEL', 'XYL', 'YUM', 'ZBRA', 'ZBH', 'ZION', 'ZTS']


[('MMM', 'Industrials'), ('AOS', 'Industrials'), ('ABT', 'Health Care'), ('ABBV', 'Health Care'), ('ACN', 'Information Technology'), ('ADBE', 'Information Technology'), ('AMD', 'Information Technology'), ('AES', 'Utilities'), ('AFL', 'Financials'), ('A', 'Health Care')]


In [ ]:
columns = ['Symbol', 'Date', 'Close', 'Volume', 'Market Cap', 'Industry']
data_df = pd.DataFrame(columns=columns)

rows_list = []
counter = 0
# Fetch historical data for each symbol and append it to the DataFrame
for symbol in sp500_symbols_hardcoded:
    counter+=1
    if (counter % 100 == 0):
      print("iteration number" + str(counter))
    stock = yf.Ticker(symbol)
    hist = stock.history(period="10y")
      # Fetch 10 years of historical data

    info = stock.info
    market_cap = info.get('marketCap', 'Unknown')
    industry = info.get('industry', 'Unknown')

    # Append each row of historical data to the DataFrame
    for date, row in hist.iterrows():
        rows_list.append({
            'Symbol': symbol,
            'Date': date,
            'Close': row['Close'],
            'Volume': row['Volume'],
            'Market Cap': market_cap,
            'Industry': industry
        })

data_df = pd.concat([data_df, pd.DataFrame(rows_list)], ignore_index=True)
print(data_df.shape)

iteration number100
iteration number200
iteration number300
iteration number400
iteration number500
(1236644, 6)


In [ ]:
def calculate_sma(data, window=20):
    """Calculates Simple Moving Average"""
    return data.rolling(window=window).mean()

def calculate_rsi(data, window=14):
    """Calculates Relative Strength Index"""
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    return 100 - (100 / (1 + rs))

In [ ]:
# Group by 'Symbol' and apply the technical indicator functions
data_df['SMA_20'] = data_df.groupby('Symbol')['Close'].apply(lambda x: calculate_sma(x, 20))
data_df['RSI_14'] = data_df.groupby('Symbol')['Close'].apply(lambda x: calculate_rsi(x, 14))

# Normalize features (example using MinMaxScaler for 'Close' and 'Volume')

scaler = MinMaxScaler()

# Note: Typically, you should fit the scaler on the training dataset only, then transform both training and test datasets.
# Here, for simplicity, we're applying it directly.
data_df[['Close', 'Volume']] = scaler.fit_transform(data_df[['Close', 'Volume']])

# Create lagged features for 'Close' and 'Volume'
def create_lagged_features(df, n_lags=5, columns=['Close', 'Volume']):
    for col in columns:
        for lag in range(1, n_lags + 1):
            df[f'{col}_lag_{lag}'] = df[col].shift(lag)
    return df

# Apply lagged feature creation to each symbol
data_df = data_df.groupby('Symbol').apply(lambda x: create_lagged_features(x, n_lags=5))

# Drop rows with NaN values that were introduced by lagging
data_df.dropna(inplace=True)

# Your data_df now has SMA, RSI, normalized 'Close' and 'Volume', and their lagged features


<ipython-input-13-1fd2a1268a99>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data_df['SMA_20'] = data_df.groupby('Symbol')['Close'].apply(lambda x: calculate_sma(x, 20))
<ipython-input-13-1fd2a1268a99>:3: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data_df['RSI_14'] = data_df.groupby('Symb

In [ ]:
# Assuming 'data_df' already has SMA, RSI, normalized 'Close' and 'Volume', and their lagged features

# Apply one-hot encoding to the 'Symbol' column to handle it as a categorical variable
data_df = pd.get_dummies(data_df, columns=['Symbol'])

# At this point, 'data_df' includes:
# - Normalized 'Close' and 'Volume'
# - SMA and RSI technical indicators
# - Lagged features for 'Close' and 'Volume'
# - One-hot encoded stock symbols

# Optionally, you can also normalize the SMA and RSI features if they haven't been normalized yet
scaler = MinMaxScaler()

# Assuming SMA and RSI might not be scaled, you can scale them here
data_df[['SMA_20', 'RSI_14']] = scaler.fit_transform(data_df[['SMA_20', 'RSI_14']])


In [ ]:
print(data_df.shape)
print(data_df.columns)

(1226787, 520)
Index(['Date', 'Close', 'Volume', 'Market Cap', 'Industry', 'SMA_20', 'RSI_14',
       'Close_lag_1', 'Close_lag_2', 'Close_lag_3',
       ...
       'Symbol_WYNN', 'Symbol_XEL', 'Symbol_XOM', 'Symbol_XRAY', 'Symbol_XYL',
       'Symbol_YUM', 'Symbol_ZBH', 'Symbol_ZBRA', 'Symbol_ZION', 'Symbol_ZTS'],
      dtype='object', length=520)


In [ ]:
"""def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

# Assuming you want to predict the next day's 'Close' price
# Let's separate the features (X) and the target (y)
X = data_df.drop(['Close'], axis=1)  # Features include all but the target 'Close'
y = data_df['Close']  # Target variable

# Number of time steps you want to look back
time_steps = 5

# Create sequences
X_seq, y_seq = create_sequences(X, y, time_steps=time_steps)

# Now, X_seq and y_seq are ready for LSTM model input"""
#issue with way to much data, perhaps choose less time, less features, and/or, less stocks


In [ ]:
dir(yf.Ticker)

In [ ]:
print(yf.data)

In [ ]:
luckinCoffee = yf.download("lkncy", start, end)
lkncy = yf.Ticker("lkncy")
aapl = yf.Ticker("aapl")
apple1m = aapl.history(interval = "1m", start = '2024-02-25', end = '2024-03-01')
print(lkncy.info)
print(apple1m.head())
print(aapl.major_holders)
print(aapl.recommendations)
#print(lkncy.major_holders)
luckinCoffee.tail(10)
##print(luckinCoffee)

In [ ]:

# Example: Predicting Luckin's stock price (dummy data and process)

data = luckinCoffee

# Preprocess data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

# Create a dataset for training
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data)-time_step-1):
        a = data[i:(i+time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 100
X, y = create_dataset(scaled_data, time_step)
X = X.reshape(X.shape[0],X.shape[1],1)

# Split data into train and test
train_size = int(len(X)*0.67)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(100, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, batch_size=1, epochs=1)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Evaluate your model (e.g., using RMSE)


In [ ]:
#print predictions
print(X_test)
print(predictions)